In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from tqdm import tqdm
import spacy
from gensim import corpora, models
import pandas as pd
import sys
sys.path.append('..')
from utils import preprocess, get_windows

In [23]:
MIN_COUNTS = 20
MAX_COUNTS = 1800
# words with count < MIN_COUNTS
# and count > MAX_COUNTS
# will be removed

MIN_LENGTH = 15
# minimum document length 
# (number of words)
# after preprocessing

# half the size of the context around a word
HALF_WINDOW_SIZE = 5
# it must be that 2*HALF_WINDOW_SIZE < MIN_LENGTH

# Load NLP model

In [24]:
nlp = spacy.load('en_core_web_sm')

# Load dataset

In [25]:
rvData = pd.read_csv("reviews.csv")
# pre-processing
rvData['cleanReviews'] = rvData['reviews'].str.replace("[^a-zA-Z#]", " ") # remove special characters
docs = rvData['reviews']

In [26]:
# number of documents
len(docs)

1867

In [27]:
# store an index with a document
docs = [(i, doc) for i, doc in enumerate(docs)]

# Preprocess dataset and create windows

In [28]:
encoded_docs, decoder, word_counts = preprocess(
    docs, nlp, MIN_LENGTH, MIN_COUNTS, MAX_COUNTS
)

100%|██████████| 1867/1867 [00:33<00:00, 56.32it/s]


number of removed short documents: 517
total number of tokens: 69224
number of tokens to be removed: 18242
number of additionally removed short documents: 200
total number of tokens: 48585

minimum word count number: 16
this number can be less than MIN_COUNTS because of document removal


In [29]:
# new ids will be created for the documents.
# create a way of restoring initial ids:
doc_decoder = {i: doc_id for i, (doc_id, doc) in enumerate(encoded_docs)}

In [30]:
data = []
# new ids are created here
for index, (_, doc) in tqdm(enumerate(encoded_docs)):
    windows = get_windows(doc, HALF_WINDOW_SIZE)
    # index represents id of a document, 
    # windows is a list of (word, window around this word),
    # where word is in the document
    data += [[index, w[0]] + w[1] for w in windows]

data = np.array(data, dtype='int64')

1150it [00:00, 9826.09it/s]


In [31]:
# a row in 'data' contains:
# id of a document, id of a word in this document, a window around this word
# 1 + 1 + 10
data.shape[1]

12

In [32]:
# number of windows (equals to the total number of tokens)
data.shape[0]

48585

# Get unigram distribution

In [33]:
word_counts = np.array(word_counts)
unigram_distribution = word_counts/sum(word_counts)

# Prepare word vectors

In [34]:
%%time
vocab_size = len(decoder)
embedding_dim = 50

# train a skip-gram word2vec model
texts = [[str(j) for j in doc] for i, doc in encoded_docs]
model = models.Word2Vec(texts, size=embedding_dim, window=5, workers=4, sg=1, negative=15, iter=70)
model.init_sims(replace=True)

word_vectors = np.zeros((vocab_size, embedding_dim)).astype('float32')
for i in decoder:
    word_vectors[i] = model.wv[str(i)]

Wall time: 17.5 s


In [35]:
# number of unique words
vocab_size

522

# Prepare initialization for document weights

In [36]:
texts = [[decoder[j] for j in doc] for i, doc in encoded_docs]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [37]:
%%time
n_topics = 25
lda = models.LdaModel(corpus, alpha=0.9, id2word=dictionary, num_topics=n_topics)
corpus_lda = lda[corpus]

Wall time: 1.15 s


In [38]:
for i, topics in lda.show_topics(n_topics, formatted=False):
    print('topic', i, ':', ' '.join([t for t, _ in topics]))

topic 0 : play family hacker get share account online item like way
topic 1 : play family get hacker item account online cheater share like
topic 2 : play family online get hacker item like share way Souls
topic 3 : play family hacker online get share account character way like
topic 4 : play family get online share hacker good like cause cheater
topic 5 : play get family item hacker share cheater like way account
topic 6 : play get family cheater way share player online item hacker
topic 7 : play family share hacker account get item time cheater player
topic 8 : play get family item hacker like account share player cheat
topic 9 : play way account item share hacker family get people cheat
topic 10 : play share hacker family get account like cheater online cause
topic 11 : play hacker share family time account item get online player
topic 12 : play get family item hacker like player way account share
topic 13 : play get family hacker good player item like share way
topic 14 : play fami

In [39]:
doc_weights_init = np.zeros((len(corpus_lda), n_topics))
for i in tqdm(range(len(corpus_lda))):
    topics = corpus_lda[i]
    for j, prob in topics:
        doc_weights_init[i, j] = prob

100%|██████████| 1150/1150 [00:00<00:00, 1838.14it/s]


# Save data

In [40]:
np.save('data.npy', data)
np.save('word_vectors.npy', word_vectors)
np.save('unigram_distribution.npy', unigram_distribution)
np.save('decoder.npy', decoder)
np.save('doc_decoder.npy', doc_decoder)
np.save('doc_weights_init.npy', doc_weights_init)